In [ ]:
# This script tries to model the cost of siphash

import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import math
from scipy import optimize
import numpy as np
from sklearn.metrics import r2_score

figsize=(4.5, 3)

TUMBlue='#3070b3'
TUMGreen='#a2ad00'
TUMOrange='#e37222'

In [ ]:
data = pd.read_csv("../data-files/siphash.csv")

In [ ]:
# The siphash model
def fitfunc(x,t,o):
    # Actual mathematical model
    def runner(a):
        return (t/a) + o
    # Magic for scipy
    if isinstance(x, np.ndarray):
        ret = []
        for s in x:
            ret += [runner(s)]
        return ret
    else:
        return runner(x)

#params = [1.02,2**11,600]
# Fit the data
params, _ = optimize.curve_fit(fitfunc, data['size'], data['cycles'])

print(params)

# Plot the data
fit = []
for s in data['size']:
    fit += [fitfunc(s,params[0],params[1])]

r2 = r2_score(data['cycles'], fit) 
print("R2 score: " + str(r2))
    
fig = plt.figure(figsize=figsize, dpi=300)

plt.plot(data['size'], fit, color=TUMOrange, label="fit", zorder=2)
plt.scatter(data['size'], data['cycles'], 5, label="data", color=TUMBlue, zorder=1)

plt.ylim([1,2**10])
#plt.semilogx(basex=2)
plt.loglog(basex=2, basey=2)
plt.grid(b=True)
plt.xlabel('\#bbox_to_anchor=(1, 1)Operations')
plt.ylabel('Cycles per Op')
plt.legend()
#fig.savefig('output/synth-siphash.pdf',bbox_inches='tight')
#plt.show()

from matplotlib2tikz import save as tikz_save
tikz_save('../tex-output/siphash.tex',
          figureheight='3.5cm',
          figurewidth='.9\\columnwidth',
          extra_axis_parameters = {"log basis x=2",
                                   "mark options={scale=.15}",
                                   "legend pos = south east"}
         )